In [24]:
%load_ext autoreload
%autoreload 2

import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

import pandas as pd

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### 时间较长，思考怎么改成多线程跑

In [25]:
# %%time
# def func(row, rc_df):
#     return len(rc_df[rc_df.index == row])

# rc_df["hcount"] = rc_df["dt_point_holding.1"].apply(func, rc_df=rc_df)
# rc_df.head()

In [26]:
BIG_TABLE_DATA_PATH="../data/yht_eur_big_table.csv"
INPUT_BAR_DATA_PATHS=["../data/EURUSD5_duka_2020.csv", "../data/EURUSD5_duka_2021.csv"]
SYMBOL="EURUSD"
OUTPUT_PATH="../data/yht_duka_5m_eur_with_floatingprofits_table.csv"

### 校验部分

In [27]:
%%time
rc_df = pd.read_csv(BIG_TABLE_DATA_PATH, index_col=["dt_point_holding"])
rc_df.head()

CPU times: user 173 ms, sys: 0 ns, total: 173 ms
Wall time: 173 ms


,symbol,action,lots,open_date,close_date,open_price,close_price,profits,pips,commission,...,profits_calc,commission_by_pips,date_var,swap_per_date_var_and_pip,open_date_mod_5,close_date_mod_5,dt_range,dt_point_holding.1,dt_range_inter_num,cur_holding_cnt
dt_point_holding,,,,,,,,,,,,,,,,,,,,,
2020-10-14 10:05:00,EURUSD,sell,0.01,2020-10-14 10:08:18,2020-10-14 12:31:20,1.17321,1.17625,-3.04,-30.4,-0.04,...,-3.08,-0.4,0,0.0,2020-10-14 10:05:00,2020-10-14 12:30:00,0 days 02:25:00,2020-10-14 10:05:00,30,1
2020-10-14 10:10:00,EURUSD,sell,0.01,2020-10-14 10:08:18,2020-10-14 12:31:20,1.17321,1.17625,-3.04,-30.4,-0.04,...,-3.08,-0.4,0,0.0,2020-10-14 10:05:00,2020-10-14 12:30:00,0 days 02:25:00,2020-10-14 10:10:00,30,1
2020-10-14 10:15:00,EURUSD,sell,0.01,2020-10-14 10:08:18,2020-10-14 12:31:20,1.17321,1.17625,-3.04,-30.4,-0.04,...,-3.08,-0.4,0,0.0,2020-10-14 10:05:00,2020-10-14 12:30:00,0 days 02:25:00,2020-10-14 10:15:00,30,1
2020-10-14 10:20:00,EURUSD,sell,0.01,2020-10-14 10:08:18,2020-10-14 12:31:20,1.17321,1.17625,-3.04,-30.4,-0.04,...,-3.08,-0.4,0,0.0,2020-10-14 10:05:00,2020-10-14 12:30:00,0 days 02:25:00,2020-10-14 10:20:00,30,1
2020-10-14 10:25:00,EURUSD,sell,0.01,2020-10-14 10:08:18,2020-10-14 12:31:20,1.17321,1.17625,-3.04,-30.4,-0.04,...,-3.08,-0.4,0,0.0,2020-10-14 10:05:00,2020-10-14 12:30:00,0 days 02:25:00,2020-10-14 10:25:00,30,2


In [28]:
%%time 
rc_df_cnt = rc_df.groupby(by=[rc_df.index], dropna=False).agg(cur_holding_cnt=("symbol", "count"))
df_merged_l = pd.merge(rc_df, rc_df_cnt, left_on="dt_point_holding", right_on="dt_point_holding", how='left')
df_merged_o = pd.merge(rc_df, rc_df_cnt, left_on="dt_point_holding", right_on="dt_point_holding", how='outer')
assert len(df_merged_l) == len(df_merged_o), (
       f"left join results should be eq to the outer join"
)
rc_df = df_merged_l
rc_df.head()

CPU times: user 81.7 ms, sys: 0 ns, total: 81.7 ms
Wall time: 81.4 ms


,symbol,action,lots,open_date,close_date,open_price,close_price,profits,pips,commission,...,commission_by_pips,date_var,swap_per_date_var_and_pip,open_date_mod_5,close_date_mod_5,dt_range,dt_point_holding.1,dt_range_inter_num,cur_holding_cnt_x,cur_holding_cnt_y
dt_point_holding,,,,,,,,,,,,,,,,,,,,,
2020-10-14 10:05:00,EURUSD,sell,0.01,2020-10-14 10:08:18,2020-10-14 12:31:20,1.17321,1.17625,-3.04,-30.4,-0.04,...,-0.4,0,0.0,2020-10-14 10:05:00,2020-10-14 12:30:00,0 days 02:25:00,2020-10-14 10:05:00,30,1,1
2020-10-14 10:10:00,EURUSD,sell,0.01,2020-10-14 10:08:18,2020-10-14 12:31:20,1.17321,1.17625,-3.04,-30.4,-0.04,...,-0.4,0,0.0,2020-10-14 10:05:00,2020-10-14 12:30:00,0 days 02:25:00,2020-10-14 10:10:00,30,1,1
2020-10-14 10:15:00,EURUSD,sell,0.01,2020-10-14 10:08:18,2020-10-14 12:31:20,1.17321,1.17625,-3.04,-30.4,-0.04,...,-0.4,0,0.0,2020-10-14 10:05:00,2020-10-14 12:30:00,0 days 02:25:00,2020-10-14 10:15:00,30,1,1
2020-10-14 10:20:00,EURUSD,sell,0.01,2020-10-14 10:08:18,2020-10-14 12:31:20,1.17321,1.17625,-3.04,-30.4,-0.04,...,-0.4,0,0.0,2020-10-14 10:05:00,2020-10-14 12:30:00,0 days 02:25:00,2020-10-14 10:20:00,30,1,1
2020-10-14 10:25:00,EURUSD,sell,0.01,2020-10-14 10:08:18,2020-10-14 12:31:20,1.17321,1.17625,-3.04,-30.4,-0.04,...,-0.4,0,0.0,2020-10-14 10:05:00,2020-10-14 12:30:00,0 days 02:25:00,2020-10-14 10:25:00,30,2,2


### Reads the input bar data, pay attention to the time slice
### Time Slice can reduce the time to process records

In [29]:
from Utils.data_get_utils import get_duka_data_df
from Utils.data_process_utils import slice_df_by_timerange
from datetime import timedelta
from datetime import datetime

from Utils.data_get_utils import get_duka_data_df_by_list
duka_5m_df = get_duka_data_df_by_list(INPUT_BAR_DATA_PATHS, SYMBOL)

In [30]:
rc_start_dt = pd.to_datetime(rc_df.index).min() - timedelta(hours=2)
rc_end_dt = pd.to_datetime(rc_df.index).max() + timedelta(hours=2)
print("slicing date range : (", rc_start_dt, ", ", rc_end_dt, ")")
duka_5m_df = slice_df_by_timerange(duka_5m_df, "datetime", rc_start_dt, rc_end_dt)

len(duka_5m_df)

slicing date range : ( 2020-10-14 08:05:00 ,  2021-07-09 05:05:00 )


54784

In [ ]:
%%time
from Utils.data_get_utils import calc_profit
import numpy as np

def calc_floating_profits(row, rc_df):
    cur_dt,low,high = row["datetime"],row["low"],row["high"]
    cur_holding_orders_df = rc_df[rc_df.index == str(cur_dt)]
    
    cur_holding_orders_df["close_price"] = np.where(cur_holding_orders_df["action"]=="buy", low, high)
    cur_holding_orders_floating_profit_ser = calc_profit(cur_holding_orders_df, check_pips_eq=False, check_output_col=False)
    return sum(cur_holding_orders_floating_profit_ser)

yht_rc_used_df = rc_df.copy()
# duka_5m_df["floating_profit"] = duka_5m_df.apply(calc_floating_profits, rc_df=yht_rc_used_df, axis=1)
duka_5m_df["floating_profit"] = duka_5m_df.apply(calc_floating_profits, rc_df=yht_rc_used_df, axis=1)
duka_5m_df.set_index("datetime",inplace=True)
duka_5m_df.sort_index(axis=0, inplace=True)

In [ ]:
duka_5m_df.to_csv(OUTPUT_PATH)

In [ ]:
duka_5m_df.head()

In [ ]:
%%time
duka_5m_df["floating_profit"].plot()

In [ ]:
duka_5m_df[duka_5m_df["floating_profit"]==duka_5m_df["floating_profit"].min()]